# Wireless FiDEATHlity

## Objective

"Suicide is one of the leading causes of death among all American adults and rates are increasing in both men and women"(SSM Health).


## Research Questions

1. Which country has the most suicides ? Which country has the least suicides ?

2. What is the relationship between gender and the number of suicides?

3. Is there a relation about people with internet access, eating disorders and depression?

4. How is the relation between internet access and suicide rates by country?

5. Which age group are more likely to suffer deaths by self harm?

### Import Libraries

In [1]:
import requests
import json
import pandas as pd

r = requests.get('http://apps.who.int/gho/athena/api/GHO/MH_12/.json?filter=Year:2019')
data = r.json()

### Convert JSON to Python list

In [2]:
list_of_data = []
for fact in data['fact']:
    for category in fact['Dim']:
        if category['category'] == 'COUNTRY':
            country_code = category['code']
        elif category['category'] == "SEX":
            sex = category['code']
    suicide_rate = fact['value']['numeric']
    list_of_data.append({'Country Code': country_code, 'Sex': sex, "Suicide Rate": suicide_rate})

country_codes = []
for country in list_of_data:
    country_codes.append(country['Country Code'])

country_codes = set(country_codes)

data_dictionary = []
for country in country_codes:
    data_dictionary.append({'Country Code': country, 'Male Suicide Rate':'', 'Female Suicide Rate':'', 'Combined Suicide Rate':''})

### Creating Python List with dictionary

In [3]:
for data in list_of_data:
    for country in data_dictionary:
        if data['Country Code'] == country['Country Code']:
            if data['Sex'] == 'MLE':
                country['Male Suicide Rate'] = data['Suicide Rate']
            elif data['Sex'] == "FMLE":
                country['Female Suicide Rate'] = data['Suicide Rate']
            elif data['Sex'] == 'BTSX':
                country['Combined Suicide Rate'] = data['Suicide Rate']

##### My first precious dataset <3

In [4]:
suicide_rates = pd.DataFrame.from_dict(data_dictionary,)
#suicide_rates.set_index(['Country Code'], inplace=True)
#suicide_rates.sort_index(inplace=True)
suicide_rates

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate
0,VUT,33.0968,9.010,20.9647
1,AFG,6.1900,5.670,5.9600
2,BRN,4.2400,0.781,2.5400
3,GNB,19.7816,6.690,12.3787
4,MYS,9.0200,2.390,5.7700
...,...,...,...,...
178,GEO,13.9698,2.170,7.6500
179,MDG,13.3322,5.420,9.1800
180,ZWE,9.6500,4.810,7.1900
181,CYP,5.3000,1.060,3.1800


### Add Country Name column

In [5]:
#!pip install pycountry
import pandas as pd
import pycountry

countries = pd.DataFrame()
data = []
for country in pycountry.countries:
    try:
        current = {'Country Code': country.alpha_3,
                    'name': country.name,
                    'official_name': country.official_name}
    except Exception as ex:
        current = {'Country Code': country.alpha_3,
                    'name': country.name,
                    'official_name': None}        
    data.append(current)

countries = pd.DataFrame(data, columns = ['Country Code', 'name', 'official_name'])
countries = countries.drop('official_name', axis=1)

In [6]:
countries

,Country Code,name
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALA,Åland Islands
...,...,...
244,WSM,Samoa
245,YEM,Yemen
246,ZAF,South Africa
247,ZMB,Zambia


In [7]:
#suicide_rate = pd.concat([suicide_rates, country], axis=1, join='inner')

suicide_rate= pd.merge(suicide_rates, countries, on='Country Code')

In [8]:
suicide_rate

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,name
0,VUT,33.0968,9.010,20.9647,Vanuatu
1,AFG,6.1900,5.670,5.9600,Afghanistan
2,BRN,4.2400,0.781,2.5400,Brunei Darussalam
3,GNB,19.7816,6.690,12.3787,Guinea-Bissau
4,MYS,9.0200,2.390,5.7700,Malaysia
...,...,...,...,...,...
178,GEO,13.9698,2.170,7.6500,Georgia
179,MDG,13.3322,5.420,9.1800,Madagascar
180,ZWE,9.6500,4.810,7.1900,Zimbabwe
181,CYP,5.3000,1.060,3.1800,Cyprus


### Check for Nans

In [9]:
suicide_rate.isna().sum()

Country Code             0
Male Suicide Rate        0
Female Suicide Rate      0
Combined Suicide Rate    0
name                     0
dtype: int64

In [10]:
suicide_rate.set_index(['Country Code'], inplace=True)
suicide_rate.sort_index(inplace=True)

### Importing percentage of population

In [11]:
r = requests.get('http://apps.who.int/gho/athena/api/GHO/AIR_4/?format=json')
data = r.json()

In [12]:
data_dictionary = []
for country in data['fact']:
    for category in country['Dim']:
        if category['category'] == "COUNTRY":
            countrycode = category['code']
    data_dictionary.append({"Country Code": countrycode, "% Living in Cities > 100k": country['value']['numeric']})

In [53]:
#data_dictionary

In [14]:
percent_living_in_cities = pd.DataFrame(data_dictionary)
percent_living_in_cities.head()

,Country Code,% Living in Cities > 100k
0,AFG,2154.64204
1,AFG,957.70975
2,AFG,1196.93229
3,AGO,2229.28417
4,AGO,947.90736


##### OOPS SEEMS LIKE WE HAVE 3 VALUES OF EVERY COUNTRY... C1+C2=C3

I have 2 drop C1 and C2 which makes the sum of C3

In [15]:
percent_living_in_cities=percent_living_in_cities.drop_duplicates(subset=['Country Code'])

In [16]:
percent_living_in_cities

,Country Code,% Living in Cities > 100k
0,AFG,2154.64204
3,AGO,2229.28417
6,ALB,3.44900
9,ARE,7.89728
12,ARG,34.85399
...,...,...
534,WSM,0.35679
537,YEM,1802.18178
540,ZAF,1116.16673
543,ZMB,618.69992


In [17]:
suicide_rate= pd.merge(percent_living_in_cities, suicide_rate, on='Country Code')
suicide_rate.shape

(183, 6)

In [18]:
suicide_rate['% Living in Cities > 100k'].fillna(suicide_rate['% Living in Cities > 100k'].median(), inplace=True)
suicide_rate.isna().sum()

Country Code                 0
% Living in Cities > 100k    0
Male Suicide Rate            0
Female Suicide Rate          0
Combined Suicide Rate        0
name                         0
dtype: int64

### Internet access per country

In [19]:
internet = pd.read_csv('/Users/user/Downloads/share-of-individuals-using-the-internet.csv')
internet

,Entity,Code,Year,Individuals using the Internet (% of population)
0,Afghanistan,AFG,1990,0.000000
1,Afghanistan,AFG,1991,0.000000
2,Afghanistan,AFG,1992,0.000000
3,Afghanistan,AFG,1993,0.000000
4,Afghanistan,AFG,1994,0.000000
...,...,...,...,...
6528,Zimbabwe,ZWE,2016,23.119989
6529,Zimbabwe,ZWE,2017,24.400000
6530,Zimbabwe,ZWE,2018,25.000000
6531,Zimbabwe,ZWE,2019,25.100000


In [20]:
internet = internet.drop(internet[internet.Year != 2019].index)

In [21]:
internet = internet.rename(columns={"Code": "Country Code"})

In [22]:
internet_suicide= pd.merge(suicide_rate, internet, on='Country Code')
internet_suicide

,Country Code,% Living in Cities > 100k,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,name,Entity,Year,Individuals using the Internet (% of population)
0,AFG,2154.64204,6.1900,5.67,5.9600,Afghanistan,Afghanistan,2019,17.600000
1,AGO,2229.28417,21.7053,4.68,12.5546,Angola,Angola,2019,32.400002
2,ALB,3.44900,5.2700,2.22,3.7200,Albania,Albania,2019,68.550392
3,ARE,7.89728,6.2900,2.63,5.2400,United Arab Emirates,United Arab Emirates,2019,99.150002
4,ARG,34.85399,13.4763,3.31,8.1400,Argentina,Argentina,2019,79.900002
...,...,...,...,...,...,...,...,...,...
148,UZB,317.69705,11.8412,4.93,8.2800,Uzbekistan,Uzbekistan,2019,70.400002
149,VNM,619.54268,10.5563,4.18,7.2200,Viet Nam,Vietnam,2019,68.661583
150,ZAF,1116.16673,37.8794,9.83,23.4856,South Africa,South Africa,2019,68.199997
151,ZMB,618.69992,25.7417,5.28,14.4258,Zambia,Zambia,2019,16.799999


In [23]:
internet_suicide=internet_suicide.drop(columns={'% Living in Cities > 100k','Entity','Year'})
internet_suicide

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,name,Individuals using the Internet (% of population)
0,AFG,6.1900,5.67,5.9600,Afghanistan,17.600000
1,AGO,21.7053,4.68,12.5546,Angola,32.400002
2,ALB,5.2700,2.22,3.7200,Albania,68.550392
3,ARE,6.2900,2.63,5.2400,United Arab Emirates,99.150002
4,ARG,13.4763,3.31,8.1400,Argentina,79.900002
...,...,...,...,...,...,...
148,UZB,11.8412,4.93,8.2800,Uzbekistan,70.400002
149,VNM,10.5563,4.18,7.2200,Viet Nam,68.661583
150,ZAF,37.8794,9.83,23.4856,South Africa,68.199997
151,ZMB,25.7417,5.28,14.4258,Zambia,16.799999


In [24]:
internet_suicide.isna().sum()

Country Code                                        0
Male Suicide Rate                                   0
Female Suicide Rate                                 0
Combined Suicide Rate                               0
name                                                0
Individuals using the Internet (% of population)    0
dtype: int64

### Eating Disorder

In [25]:
eatingdisorder = pd.read_csv('/Users/user/Downloads/share-with-an-eating-disorder.csv')
eatingdisorder

,Entity,Code,Year,Prevalence - Eating disorders - Sex: Both - Age: Age-standardized (Percent)
0,Afghanistan,AFG,1990,0.13
1,Afghanistan,AFG,1991,0.13
2,Afghanistan,AFG,1992,0.12
3,Afghanistan,AFG,1993,0.12
4,Afghanistan,AFG,1994,0.11
...,...,...,...,...
6835,Zimbabwe,ZWE,2015,0.10
6836,Zimbabwe,ZWE,2016,0.10
6837,Zimbabwe,ZWE,2017,0.10
6838,Zimbabwe,ZWE,2018,0.10


In [26]:
eatingdisorder = eatingdisorder.rename(columns={'Prevalence - Eating disorders - Sex: Both - Age: Age-standardized (Percent)':'Eating Disorder(%)','Code':'Country Code'})
eatingdisorder

,Entity,Country Code,Year,Eating Disorder(%)
0,Afghanistan,AFG,1990,0.13
1,Afghanistan,AFG,1991,0.13
2,Afghanistan,AFG,1992,0.12
3,Afghanistan,AFG,1993,0.12
4,Afghanistan,AFG,1994,0.11
...,...,...,...,...
6835,Zimbabwe,ZWE,2015,0.10
6836,Zimbabwe,ZWE,2016,0.10
6837,Zimbabwe,ZWE,2017,0.10
6838,Zimbabwe,ZWE,2018,0.10


In [27]:
eatingdisorder = eatingdisorder.drop(eatingdisorder[eatingdisorder.Year != 2019].index)
eatingdisorder

,Entity,Country Code,Year,Eating Disorder(%)
29,Afghanistan,AFG,2019,0.12
59,African Region (WHO),NaN,2019,0.11
89,Albania,ALB,2019,0.15
119,Algeria,DZA,2019,0.22
149,American Samoa,ASM,2019,0.13
...,...,...,...,...
6719,World Bank Lower Middle Income,NaN,2019,0.13
6749,World Bank Upper Middle Income,NaN,2019,0.18
6779,Yemen,YEM,2019,0.13
6809,Zambia,ZMB,2019,0.12


In [28]:
internet_suicide_eatingdisorder= pd.merge(internet_suicide, eatingdisorder, on='Country Code')
internet_suicide_eatingdisorder

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,name,Individuals using the Internet (% of population),Entity,Year,Eating Disorder(%)
0,AFG,6.1900,5.67,5.9600,Afghanistan,17.600000,Afghanistan,2019,0.12
1,AGO,21.7053,4.68,12.5546,Angola,32.400002,Angola,2019,0.14
2,ALB,5.2700,2.22,3.7200,Albania,68.550392,Albania,2019,0.15
3,ARE,6.2900,2.63,5.2400,United Arab Emirates,99.150002,United Arab Emirates,2019,0.32
4,ARG,13.4763,3.31,8.1400,Argentina,79.900002,Argentina,2019,0.36
...,...,...,...,...,...,...,...,...,...
148,UZB,11.8412,4.93,8.2800,Uzbekistan,70.400002,Uzbekistan,2019,0.12
149,VNM,10.5563,4.18,7.2200,Viet Nam,68.661583,Vietnam,2019,0.10
150,ZAF,37.8794,9.83,23.4856,South Africa,68.199997,South Africa,2019,0.17
151,ZMB,25.7417,5.28,14.4258,Zambia,16.799999,Zambia,2019,0.12


In [29]:
internet_suicide_eatingdisorder.isna().sum()

Country Code                                        0
Male Suicide Rate                                   0
Female Suicide Rate                                 0
Combined Suicide Rate                               0
name                                                0
Individuals using the Internet (% of population)    0
Entity                                              0
Year                                                0
Eating Disorder(%)                                  0
dtype: int64

In [30]:
internet_suicide_eatingdisorder = internet_suicide_eatingdisorder.drop(columns={'name','Year'})
internet_suicide_eatingdisorder

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,Individuals using the Internet (% of population),Entity,Eating Disorder(%)
0,AFG,6.1900,5.67,5.9600,17.600000,Afghanistan,0.12
1,AGO,21.7053,4.68,12.5546,32.400002,Angola,0.14
2,ALB,5.2700,2.22,3.7200,68.550392,Albania,0.15
3,ARE,6.2900,2.63,5.2400,99.150002,United Arab Emirates,0.32
4,ARG,13.4763,3.31,8.1400,79.900002,Argentina,0.36
...,...,...,...,...,...,...,...
148,UZB,11.8412,4.93,8.2800,70.400002,Uzbekistan,0.12
149,VNM,10.5563,4.18,7.2200,68.661583,Vietnam,0.10
150,ZAF,37.8794,9.83,23.4856,68.199997,South Africa,0.17
151,ZMB,25.7417,5.28,14.4258,16.799999,Zambia,0.12


### Depression Rate by country

In [31]:
depression = pd.read_csv('/Users/user/Downloads/share-with-depression.csv')
depression

,Entity,Code,Year,Prevalence - Depressive disorders - Sex: Both - Age: Age-standardized (Percent)
0,Afghanistan,AFG,1990,5.13
1,Afghanistan,AFG,1991,5.12
2,Afghanistan,AFG,1992,5.11
3,Afghanistan,AFG,1993,5.10
4,Afghanistan,AFG,1994,5.10
...,...,...,...,...
6747,Zimbabwe,ZWE,2015,3.55
6748,Zimbabwe,ZWE,2016,3.56
6749,Zimbabwe,ZWE,2017,3.56
6750,Zimbabwe,ZWE,2018,3.56


In [32]:
depression = depression.rename(columns={'Prevalence - Depressive disorders - Sex: Both - Age: Age-standardized (Percent)':'Depressive Disorder (%)','Code':'Country Code'})
depression

,Entity,Country Code,Year,Depressive Disorder (%)
0,Afghanistan,AFG,1990,5.13
1,Afghanistan,AFG,1991,5.12
2,Afghanistan,AFG,1992,5.11
3,Afghanistan,AFG,1993,5.10
4,Afghanistan,AFG,1994,5.10
...,...,...,...,...
6747,Zimbabwe,ZWE,2015,3.55
6748,Zimbabwe,ZWE,2016,3.56
6749,Zimbabwe,ZWE,2017,3.56
6750,Zimbabwe,ZWE,2018,3.56


In [33]:
depression = depression.drop(depression[depression.Year != 2019].index)
depression

,Entity,Country Code,Year,Depressive Disorder (%)
29,Afghanistan,AFG,2019,5.13
59,African Region (WHO),NaN,2019,4.59
89,Albania,ALB,2019,2.63
119,Algeria,DZA,2019,4.37
149,American Samoa,ASM,2019,2.80
...,...,...,...,...
6631,World Bank Lower Middle Income,NaN,2019,3.72
6661,World Bank Upper Middle Income,NaN,2019,3.32
6691,Yemen,YEM,2019,5.04
6721,Zambia,ZMB,2019,4.30


In [34]:
depression = depression.drop(columns={'Entity','Year'})
depression

,Country Code,Depressive Disorder (%)
29,AFG,5.13
59,NaN,4.59
89,ALB,2.63
119,DZA,4.37
149,ASM,2.80
...,...,...
6631,NaN,3.72
6661,NaN,3.32
6691,YEM,5.04
6721,ZMB,4.30


### Final Data

In [35]:
data= pd.merge(internet_suicide_eatingdisorder, depression, on='Country Code')
data

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,Individuals using the Internet (% of population),Entity,Eating Disorder(%),Depressive Disorder (%)
0,AFG,6.1900,5.67,5.9600,17.600000,Afghanistan,0.12,5.13
1,AGO,21.7053,4.68,12.5546,32.400002,Angola,0.14,5.86
2,ALB,5.2700,2.22,3.7200,68.550392,Albania,0.15,2.63
3,ARE,6.2900,2.63,5.2400,99.150002,United Arab Emirates,0.32,3.76
4,ARG,13.4763,3.31,8.1400,79.900002,Argentina,0.36,2.59
...,...,...,...,...,...,...,...,...
146,UZB,11.8412,4.93,8.2800,70.400002,Uzbekistan,0.12,3.34
147,VNM,10.5563,4.18,7.2200,68.661583,Vietnam,0.10,2.58
148,ZAF,37.8794,9.83,23.4856,68.199997,South Africa,0.17,4.45
149,ZMB,25.7417,5.28,14.4258,16.799999,Zambia,0.12,4.30


##### I decide to remove the column % Living in Cities > 100k_y and just use the total number of population

In [36]:
population = pd.read_csv('/Users/user/Downloads/population-since-1800.csv')
population = population.drop(population[population.Year != 2019].index)
population

,Entity,Code,Year,Population (historical estimates)
219,Afghanistan,AFG,2019,37769496
441,Africa,NaN,2019,1327701380
663,Albania,ALB,2019,2873883
885,Algeria,DZA,2019,42705372
972,American Samoa,ASM,2019,47341
...,...,...,...,...
48511,Western Sahara,ESH,2019,544883
48733,World,OWID_WRL,2019,7764951040
48955,Yemen,YEM,2019,31546694
49177,Zambia,ZMB,2019,18380478


In [37]:
population = population.rename(columns={'Population (historical estimates)':'Population','Code':'Country Code'})
population

,Entity,Country Code,Year,Population
219,Afghanistan,AFG,2019,37769496
441,Africa,NaN,2019,1327701380
663,Albania,ALB,2019,2873883
885,Algeria,DZA,2019,42705372
972,American Samoa,ASM,2019,47341
...,...,...,...,...
48511,Western Sahara,ESH,2019,544883
48733,World,OWID_WRL,2019,7764951040
48955,Yemen,YEM,2019,31546694
49177,Zambia,ZMB,2019,18380478


In [38]:
population = population.drop(columns={'Entity','Year'})
population

,Country Code,Population
219,AFG,37769496
441,NaN,1327701380
663,ALB,2873883
885,DZA,42705372
972,ASM,47341
...,...,...
48511,ESH,544883
48733,OWID_WRL,7764951040
48955,YEM,31546694
49177,ZMB,18380478


In [39]:
data2= pd.merge(data, population, on='Country Code')
data2

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,Individuals using the Internet (% of population),Entity,Eating Disorder(%),Depressive Disorder (%),Population
0,AFG,6.1900,5.67,5.9600,17.600000,Afghanistan,0.12,5.13,37769496
1,AGO,21.7053,4.68,12.5546,32.400002,Angola,0.14,5.86,32353592
2,ALB,5.2700,2.22,3.7200,68.550392,Albania,0.15,2.63,2873883
3,ARE,6.2900,2.63,5.2400,99.150002,United Arab Emirates,0.32,3.76,9211660
4,ARG,13.4763,3.31,8.1400,79.900002,Argentina,0.36,2.59,44745516
...,...,...,...,...,...,...,...,...,...
146,UZB,11.8412,4.93,8.2800,70.400002,Uzbekistan,0.12,3.34,32976946
147,VNM,10.5563,4.18,7.2200,68.661583,Vietnam,0.10,2.58,95776712
148,ZAF,37.8794,9.83,23.4856,68.199997,South Africa,0.17,4.45,58087060
149,ZMB,25.7417,5.28,14.4258,16.799999,Zambia,0.12,4.30,18380478


#### oopss. I need to add the column of % of the people who live in the urban areas

In [40]:
urban_pop = pd.read_csv('/Users/user/Downloads/urban-vs-rural-majority.csv')
urban_pop = urban_pop.drop(urban_pop[urban_pop.Year != 2019].index)
urban_pop

,Entity,Code,Year,Urban population (%) long-run with 2050 projections (OWID),Rural population (%) long-run with 2050 projections (OWID)
69,Afghanistan,AFG,2019,25.75400,74.246002
177,Africa,NaN,2019,42.98265,57.017349
278,Albania,ALB,2019,61.22900,38.771000
379,Algeria,DZA,2019,73.18900,26.811001
480,American Samoa,ASM,2019,87.14700,12.853000
...,...,...,...,...,...
27536,Western Sahara,ESH,2019,86.75500,13.245000
27642,World,OWID_WRL,2019,55.73136,44.268639
27743,Yemen,YEM,2019,37.27300,62.727001
27844,Zambia,ZMB,2019,44.07200,55.928001


In [41]:
urban_pop = urban_pop.rename(columns={'Code':'Country Code','Urban population (%) long-run with 2050 projections (OWID)':'Urban Population'})
urban_pop = urban_pop.drop(columns={'Entity','Year','Rural population (%) long-run with 2050 projections (OWID)'})

In [42]:
data2= pd.merge(data2, urban_pop, on='Country Code')
data2

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,Individuals using the Internet (% of population),Entity,Eating Disorder(%),Depressive Disorder (%),Population,Urban Population
0,AFG,6.1900,5.67,5.9600,17.600000,Afghanistan,0.12,5.13,37769496,25.754
1,AGO,21.7053,4.68,12.5546,32.400002,Angola,0.14,5.86,32353592,66.177
2,ALB,5.2700,2.22,3.7200,68.550392,Albania,0.15,2.63,2873883,61.229
3,ARE,6.2900,2.63,5.2400,99.150002,United Arab Emirates,0.32,3.76,9211660,86.789
4,ARG,13.4763,3.31,8.1400,79.900002,Argentina,0.36,2.59,44745516,91.991
...,...,...,...,...,...,...,...,...,...,...
146,UZB,11.8412,4.93,8.2800,70.400002,Uzbekistan,0.12,3.34,32976946,50.433
147,VNM,10.5563,4.18,7.2200,68.661583,Vietnam,0.10,2.58,95776712,36.628
148,ZAF,37.8794,9.83,23.4856,68.199997,South Africa,0.17,4.45,58087060,66.856
149,ZMB,25.7417,5.28,14.4258,16.799999,Zambia,0.12,4.30,18380478,44.072


#### Add another data
I decide to add another variable which will be helpful on my exploratory analysis

In [43]:
data3= pd.read_csv('/Users/user/Downloads/suicide-rates-by-age-detailed.csv')
data3 = data3.drop(data3[data3.Year != 2019].index)
data3

,Entity,Code,Year,Deaths - Self-harm - Sex: Both - Age: 70+ years (Rate),Deaths - Self-harm - Sex: Both - Age: 50-69 years (Rate),Deaths - Self-harm - Sex: Both - Age: All Ages (Rate),Deaths - Self-harm - Sex: Both - Age: 5-14 years (Rate),Deaths - Self-harm - Sex: Both - Age: 15-49 years (Rate)
29,Afghanistan,AFG,2019,9.47,9.87,4.21,0.22,7.22
59,African Region (WHO),NaN,2019,44.69,20.78,6.31,0.57,7.48
89,Albania,ALB,2019,8.45,5.40,5.59,0.66,6.99
119,Algeria,DZA,2019,5.68,3.51,3.62,0.27,5.31
149,American Samoa,ASM,2019,12.38,7.46,7.12,0.52,10.66
...,...,...,...,...,...,...,...,...
6719,World Bank Lower Middle Income,NaN,2019,20.53,13.28,9.59,0.76,13.11
6749,World Bank Upper Middle Income,NaN,2019,27.44,11.83,9.10,0.47,8.68
6779,Yemen,YEM,2019,8.11,8.09,4.95,0.39,8.25
6809,Zambia,ZMB,2019,58.16,30.07,7.66,0.72,9.32


In [44]:
data3 = data3.rename(columns={'Code':'Country Code','Deaths - Self-harm - Sex: Both - Age: 70+ years (Rate)':'Deaths Age 70+','Deaths - Self-harm - Sex: Both - Age: 5-14 years (Rate)':'Deaths Age 5-14','Deaths - Self-harm - Sex: Both - Age: 50-69 years (Rate)':'Deaths Age 50-59','Deaths - Self-harm - Sex: Both - Age: 15-49 years (Rate)':'Deaths Age 15-49'})
data3 = data3.drop(columns={'Entity','Year','Deaths - Self-harm - Sex: Both - Age: All Ages (Rate)'})
data3

,Country Code,Deaths Age 70+,Deaths Age 50-59,Deaths Age 5-14,Deaths Age 15-49
29,AFG,9.47,9.87,0.22,7.22
59,NaN,44.69,20.78,0.57,7.48
89,ALB,8.45,5.40,0.66,6.99
119,DZA,5.68,3.51,0.27,5.31
149,ASM,12.38,7.46,0.52,10.66
...,...,...,...,...,...
6719,NaN,20.53,13.28,0.76,13.11
6749,NaN,27.44,11.83,0.47,8.68
6779,YEM,8.11,8.09,0.39,8.25
6809,ZMB,58.16,30.07,0.72,9.32


In [45]:
data3= pd.merge(data2, data3, on='Country Code')
data3

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,Individuals using the Internet (% of population),Entity,Eating Disorder(%),Depressive Disorder (%),Population,Urban Population,Deaths Age 70+,Deaths Age 50-59,Deaths Age 5-14,Deaths Age 15-49
0,AFG,6.1900,5.67,5.9600,17.600000,Afghanistan,0.12,5.13,37769496,25.754,9.47,9.87,0.22,7.22
1,AGO,21.7053,4.68,12.5546,32.400002,Angola,0.14,5.86,32353592,66.177,55.49,26.96,0.64,7.81
2,ALB,5.2700,2.22,3.7200,68.550392,Albania,0.15,2.63,2873883,61.229,8.45,5.40,0.66,6.99
3,ARE,6.2900,2.63,5.2400,99.150002,United Arab Emirates,0.32,3.76,9211660,86.789,7.63,8.34,0.09,8.21
4,ARG,13.4763,3.31,8.1400,79.900002,Argentina,0.36,2.59,44745516,91.991,17.95,13.50,0.72,14.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,UZB,11.8412,4.93,8.2800,70.400002,Uzbekistan,0.12,3.34,32976946,50.433,12.30,12.68,1.95,16.55
147,VNM,10.5563,4.18,7.2200,68.661583,Vietnam,0.10,2.58,95776712,36.628,23.66,14.28,0.23,8.58
148,ZAF,37.8794,9.83,23.4856,68.199997,South Africa,0.17,4.45,58087060,66.856,20.31,15.75,0.02,20.27
149,ZMB,25.7417,5.28,14.4258,16.799999,Zambia,0.12,4.30,18380478,44.072,58.16,30.07,0.72,9.32


### Clean the Final Data

I decide to drop the population column and just focus on the Urban population considering that the urban population are the ones who has internet connection

In [46]:
data3 = data3.drop(columns={'Population'})
data3

,Country Code,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate,Individuals using the Internet (% of population),Entity,Eating Disorder(%),Depressive Disorder (%),Urban Population,Deaths Age 70+,Deaths Age 50-59,Deaths Age 5-14,Deaths Age 15-49
0,AFG,6.1900,5.67,5.9600,17.600000,Afghanistan,0.12,5.13,25.754,9.47,9.87,0.22,7.22
1,AGO,21.7053,4.68,12.5546,32.400002,Angola,0.14,5.86,66.177,55.49,26.96,0.64,7.81
2,ALB,5.2700,2.22,3.7200,68.550392,Albania,0.15,2.63,61.229,8.45,5.40,0.66,6.99
3,ARE,6.2900,2.63,5.2400,99.150002,United Arab Emirates,0.32,3.76,86.789,7.63,8.34,0.09,8.21
4,ARG,13.4763,3.31,8.1400,79.900002,Argentina,0.36,2.59,91.991,17.95,13.50,0.72,14.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,UZB,11.8412,4.93,8.2800,70.400002,Uzbekistan,0.12,3.34,50.433,12.30,12.68,1.95,16.55
147,VNM,10.5563,4.18,7.2200,68.661583,Vietnam,0.10,2.58,36.628,23.66,14.28,0.23,8.58
148,ZAF,37.8794,9.83,23.4856,68.199997,South Africa,0.17,4.45,66.856,20.31,15.75,0.02,20.27
149,ZMB,25.7417,5.28,14.4258,16.799999,Zambia,0.12,4.30,44.072,58.16,30.07,0.72,9.32


In [47]:
final_df = data3.rename(columns={'Individuals using the Internet (% of population)':'Internet Access (%)'})

In [48]:
final_df = final_df[['Country Code','Entity','Urban Population','Internet Access (%)','Eating Disorder(%)','Depressive Disorder (%)','Deaths Age 5-14','Deaths Age 15-49','Deaths Age 50-59','Deaths Age 70+', 'Male Suicide Rate', 'Female Suicide Rate','Combined Suicide Rate']]
final_df = final_df.set_index('Country Code')

In [49]:
final_df

,Entity,Urban Population,Internet Access (%),Eating Disorder(%),Depressive Disorder (%),Deaths Age 5-14,Deaths Age 15-49,Deaths Age 50-59,Deaths Age 70+,Male Suicide Rate,Female Suicide Rate,Combined Suicide Rate
Country Code,,,,,,,,,,,,
AFG,Afghanistan,25.754,17.600000,0.12,5.13,0.22,7.22,9.87,9.47,6.1900,5.67,5.9600
AGO,Angola,66.177,32.400002,0.14,5.86,0.64,7.81,26.96,55.49,21.7053,4.68,12.5546
ALB,Albania,61.229,68.550392,0.15,2.63,0.66,6.99,5.40,8.45,5.2700,2.22,3.7200
ARE,United Arab Emirates,86.789,99.150002,0.32,3.76,0.09,8.21,8.34,7.63,6.2900,2.63,5.2400
ARG,Argentina,91.991,79.900002,0.36,2.59,0.72,14.20,13.50,17.95,13.4763,3.31,8.1400
...,...,...,...,...,...,...,...,...,...,...,...,...
UZB,Uzbekistan,50.433,70.400002,0.12,3.34,1.95,16.55,12.68,12.30,11.8412,4.93,8.2800
VNM,Vietnam,36.628,68.661583,0.10,2.58,0.23,8.58,14.28,23.66,10.5563,4.18,7.2200
ZAF,South Africa,66.856,68.199997,0.17,4.45,0.02,20.27,15.75,20.31,37.8794,9.83,23.4856


In [52]:
final_df.to_csv('df_project.csv')